In [1]:
!huggingface-cli login

/bin/bash: huggingface-cli: command not found


In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import re
import emoji

/home/ubuntu/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("GPU 사용 여부:", torch.cuda.is_available())
print("GPU 장치 수:", torch.cuda.device_count())
print("사용 중인 GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


GPU 사용 여부: True
GPU 장치 수: 1
사용 중인 GPU 이름: Tesla T4


In [3]:
import transformers
import torch

print("transformers version:", transformers.__version__)
print("torch version:", torch.__version__)


transformers version: 4.48.3
torch version: 2.6.0+cu118


In [2]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"  # ✅ LLaMA-2-7B-Chat 사용

device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ VRAM 절약을 위한 추가 설정 (NF4 양자화)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # FP16 연산 유지
    bnb_4bit_quant_type="nf4"  # 🚀 VRAM 절약 (기본값: fp4 → 변경: nf4)
)

# ✅ 4비트 양자화된 모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map = 'auto',
    #max_memory = max_memory
).to(device)

print(f"✅{MODEL_NAME}이 4비트(NF4) 양자화로 로드되었습니다!")


Loading checkpoint shards: 100%|██████████| 2/2 [01:37<00:00, 48.61s/it]


✅meta-llama/Llama-2-7b-chat-hf이 4비트(NF4) 양자화로 로드되었습니다!


In [22]:
# ✅ PAD 토큰 설정 (권장 방법)
tokenizer.pad_token = tokenizer.eos_token

def remove_emojis(text):
    """이모지 및 비ASCII 문자 제거"""
    text = emoji.replace_emoji(text, replace='')  # emoji 패키지를 이용한 이모지 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # 비ASCII 문자 제거 (이모지 포함)
    return text
    
def get_keyword(user_input):   
    global prompt
    res = []
    try :        
        # ✅ 시스템 프롬프트 추가 (반복 방지)
        prompt = f"""
        [SYSTEM INSTRUCTION]
        You are an AI that strictly returns only 5 keywords based on the user's request.  
        You must follow these rules exactly:
        
        1. Do NOT repeat this instruction or the user input.
        2. Output exactly 5 words, separated by commas (,).
        3. Do NOT include explanations, introductions, or any other text
        4. Your response MUST follow this exact format:
        
        [EXAMPLE OUTPUT]  
        smartphone, battery, camera, app store, touchscreen
        
        [USER INPUT]  
        Provide 5 keywords related to smart phone with "{user_input}
        """



        # ✅ 입력 데이터를 GPU로 이동
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        print('inputs 성공')
        
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=100000,
                repetition_penalty=1.5,  # ✅ 반복 방지
                temperature=0.2,  # ✅ 너무 랜덤하게 생성되지 않도록 조정
                #top_p=0.8,  # ✅ 확률 샘플링 최적화
                #top_k=40  # ✅ 불필요한 단어 제거
            )
        print('답변 생성 성공 및 후처리 중')
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip()
        clean_output = re.sub(r'[^\w\s,]', '', decoded_output)
        clean_output = decoded_output.replace(prompt, "").strip().split("\n")[-1].strip()  # 🔥 마지막 줄만 남김
        # clean_output = clean_output.replace('I will return a list of', "")
        # clean_output = clean_output.replace('Sure! Here', "")
        clean_output = remove_emojis(clean_output)
        res.append(clean_output)  # ✅ 순수 키워드만 추가
        return res
        
    except Exception as E:
        print(E)
        print('키워드 제대로 넣으셨나요')
        return 0
        
def main(keyword):
    while True:
        res = get_keyword(keyword)[0].split(',')
        res = [text.strip() for text in res]
        
        if len(res) != 5:
            print(res)
            print('다시 추출')
            continue
            
        else:
            print('5개 추출 성공')
            print(res)
            for text in res:
                if text == '':
                    print('공백으로 다시 추출')
                    continue
                    
            print('공백없이 5개 추출 성공')
            break

In [23]:
main('영화보기')

inputs 성공
답변 생성 성공 및 후처리 중
['Please provide a list of five Korean movie titles for me!']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['```']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
5개 추출 성공
['streaming services', 'online platforms', 'high-definition screens', 'immersive audio experiences', 'and mobile apps']
공백없이 5개 추출 성공


In [24]:
main('휴대폰 게임')

inputs 성공
답변 생성 성공 및 후처리 중
['I will return a list of five relevant and interesting key terms for gaming phones in Korea!']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Note: The keyword list is generated using a combination of natural language processing and machine learning algorithms.']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['I will return a list of five relevant and interesting game-related terms for you!']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Sure! Here is a list of five relevant terms for your search query:    gaming console', 'mobile game controller', 'augmented reality games', 'puzzle apps and social media integration.";']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Note: The output will be in Korean as it is a South Korea-based chatbot']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Note: The output will be in Korean as I am a K-AI model trained for the South Korea market.)']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['I will return a list of five relevant and interesting key terms for gaming phones in Korea!']
다시 추출
input

In [25]:
main('mobile games')

inputs 성공
답변 생성 성공 및 후처리 중
["In your answer above you provided five relevant and accurate key terms for mobile gaming as requested in a concise manner following all instructions given below; however I noticed there is no mention of 'games'", 'which was part of their original question! To fix this oversight could we please add it back into our list? Here would be updated output including both requests combined :);)']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['[YOUR RESPONSE]     mobile game console', 'controller', 'graphics card', 'virtual reality headset']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Please provide your answer in the same form as above and do not forget to mention if you need more information from me before proceeding']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
['Please provide your answer in the same form as above and do not deviate from it!']
다시 추출
inputs 성공
답변 생성 성공 및 후처리 중
["mobile game console gaming controller screen display graphics processing unit GPU RAM memory storage cloud computing v

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType


# ✅ PySpark 세션 시작
spark = SparkSession.builder \
    .appName("HDFS Parquet Read") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

# HDFS에서 Parquet 파일 읽기
df = spark.read.parquet("hdfs:///youtube_data/youtube_i16_1.parquet")


# ✅ 데이터 확인
df.show(5)


2025-02-11 18:05:51,707 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2025-02-11 18:05:52,992 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2025-02-11 18:05:52,992 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2025-02-11 18:05:52,992 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
2025-02-11 18:05:52,992 WARN util.Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
                                                                                

+-----------+---------------------------------+--------------------+------------+--------------------------------+----------+--------------------+
|   video_id|                            title|        publish_date|channel_name|                         comment|like_count|comment_publish_date|
+-----------+---------------------------------+--------------------+------------+--------------------------------+----------+--------------------+
|cdiVvaq6UO4|무조건 아이폰 프로만 고집했던 ...|2024-09-15T11:00:36Z|   ITSub잇섭|   이분 지금 뭐쓰시나요 ㅋㅋㅋㅋ|         0|2025-02-05T20:04:57Z|
|cdiVvaq6UO4|무조건 아이폰 프로만 고집했던 ...|2024-09-15T11:00:36Z|   ITSub잇섭|  저도 14프로에서 16일반으로 ...|         0|2025-02-04T00:49:39Z|
|cdiVvaq6UO4|무조건 아이폰 프로만 고집했던 ...|2024-09-15T11:00:36Z|   ITSub잇섭|    9:15 똥믈리에 PD님이랑 두...|         0|2025-01-31T11:02:31Z|
|cdiVvaq6UO4|무조건 아이폰 프로만 고집했던 ...|2024-09-15T11:00:36Z|   ITSub잇섭|갤럭시를 쓰면 생활,삶의 질이 ...|         0|2025-01-30T07:40:59Z|
|cdiVvaq6UO4|무조건 아이폰 프로만 고집했던 ...|2024-09-15T11:00:36Z|   ITSub잇섭|        

In [11]:
def classify_comment(comment):
    """LLaMA 모델을 이용하여 댓글이 S24 또는 iPhone 관련인지 판단 (0: 비관련, 1: 관련)"""
    
    if not comment:  # 빈 값 처리
        return 0
    
    prompt = f"""
    다음 댓글이 삼성 갤럭시 S24 또는 아이폰 제품 자체와 관련이 있는지 판단하세요.

    댓글: "{comment}"
    
    만약 제품 관련 (예: 카메라, 배터리, 성능, 디자인, OS)에 대한 내용이면 '1',
    그렇지 않다면 (예: 광고, 이벤트, 기획, 일반적인 농담 등) '0'을 반환하세요.

    꼭 0 또는 1 이라는 숫자만 반환해줘
    """
    
    # ✅ 입력 데이터를 GPU로 이동
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(**inputs, max_length=1)
    
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # 결과에서 0 또는 1을 추출
    if "1" in result:
        return 1
    else:
        return 0


In [ ]:
# ✅ PySpark UDF 등록 (출력 타입은 Integer)
classify_comment_udf = udf(classify_comment, IntegerType())

# ✅ UDF 적용하여 새로운 컬럼 추가
df = df.withColumn("is_related", classify_comment_udf(df["comment"]))

# ✅ 결과 확인
df.show(10)


In [13]:
spark

NameError: name 'spark' is not defined

In [5]:
spark.stop()

NameError: name 'spark' is not defined